In [3]:
#%pip install pystardog
#%pip install dotenv
#%pip install Unidecode
#%pip install langdetect
#%pip install -U pip setuptools wheel
#%pip install -U spacy
#!python -m spacy download en_core_web_trf

In [46]:
import stardog
import os
import pandas as pd
import io
from unidecode import unidecode
import re
from langdetect import detect
import spacy
import requests, uuid, json
import time
import json
from dotenv import load_dotenv

### Specify Stardog connection details

In [2]:
# Load environment variables from a .env file
load_dotenv()

# Stardog variables
STARDOG_ENDPOINT = os.getenv('STARDOG_ENDPOINT')
STARDOG_USERNAME = os.getenv("STARDOG_USERNAME")
STARDOG_PASSWORD = os.getenv("STARDOG_PASSWORD")

connection_details = {
    'endpoint': STARDOG_ENDPOINT,
    'username': STARDOG_USERNAME,
    'password': STARDOG_PASSWORD
}

### Create a new database in Stardog

Drop the database if it already exists.

In [3]:
#database_name = 'IndigenousSlavery'

#with stardog.Admin(**connection_details) as admin:
#   if database_name in [db.name for db in admin.databases()]:
#       admin.database(database_name).drop()
#   db = admin.new_database(database_name)

### Add the database schema

In [4]:
####
# Opcao - Adicionar o schema via Stardog Studio

### Add data to the database

In [4]:
# Lendo os dados do arquivo CSV
data = pd.read_csv("data/BDTD_Consolidado_feb2025.csv")
data.head()

,Autor(a),ID Lattes do(a) autor(a),Orientadores,ID Lattes dos orientadores,Membros da banca,ID Lattes dos membros da banca,Título,Ano de defesa,Instituição de defesa,Sigla da instituição de defesa,...,Tipos de acesso,Tipo de documento,Assuntos em português,Assuntos em inglês,Idioma,Resumo,Link de acesso,Resumo em Português,Resumo em Inglês,Referência Bibliográfica
0,LUCAS GROTH PEREIRA,Não informado pela instituição,[],Não informado pela instituição,[],Não informado pela instituição,[en] AN ABOLICIONIST ARGUMENT: PENAL ABOLITION...,2022,Pontifícia Universidade Católica do Rio de Jan...,PUC_RIO,...,openAccess,doctoralThesis,"['[pt] MARXISMO', '[pt] SISTEMA PUNITIVO', '[p...",['Não informado pela instituição'],por,[pt] O presente trabalho busca fazer uma retom...,https://www.maxwell.vrac.puc-rio.br/colecao.ph...,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição
1,"Góes, Carolina Braz, 1988-",Não informado pela instituição,[],Não informado pela instituição,[],Não informado pela instituição,Percepções dos pesquisadores da produção anima...,2019,Universidade Estadual de Campinas (UNICAMP),UNICAMP,...,openAccess,masterThesis,"['Agropecuária', 'Indústria animal', 'Animalis...",['Não informado pela instituição'],por,NaN,https://hdl.handle.net/20.500.12733/1637478||G...,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição
2,"Krob, Bruna Emerim",Não informado pela instituição,"['Xavier, Regina Célia Lima']",Não informado pela instituição,[],Não informado pela instituição,"""Com a condição de servir gratuitamente a mim ...",2016,Universidade Federal do Rio Grande do Sul (UFRGS),UFRGS,...,openAccess,masterThesis,"['Escravidão', 'Abolição da escravatura', 'Car...","['Slavery', 'Manumission', 'Abolition', 'Freed...",por,A presente pesquisa parte da emancipação de es...,http://hdl.handle.net/10183/148467,Não informado pela instituição,Não informado pela instituição,Não informado pela instituição
3,"Pessoa, Valdirene Nunes de Santana",http://lattes.cnpq.br/4782270039388546,"['Costa, Carlos Eduardo Coutinho da']",Não informado pela instituição,"['Nascimento, Álvaro Pereira do', 'Silva, Alex...",Não informado pela instituição,Entre as pessoas mais gradas do município: a t...,2018,Universidade Federal Rural do Rio de Janeiro (...,UFRRJ,...,openAccess,masterThesis,"['Negro', 'Pós-abolição', 'Trajetória', 'Mobil...","['Post-abolition', 'Trajectory', 'Social Mobil...",por,O objeto central que baliza esta pesquisa é a ...,https://rima.ufrrj.br/jspui/handle/20.500.1440...,Não informado pela instituição,Não informado pela instituição,"PESSOA, Valdirene Nunes de Santana. Entre as p..."
4,"Cruz, Itan",http://lattes.cnpq.br/7904209184794040,"['Negro, Antonio Luigi']",http://lattes.cnpq.br/8888819051963884||Não in...,"['Negro, Antonio Luigi', 'Mata, Iacy Maia', 'P...",http://lattes.cnpq.br/8888819051963884||http:/...,"Saraiva, Dantas e Cotegipe: baianismo, escravi...",2022,Universidade Federal da Bahia (UFBA),UFBA,...,openAccess,doctoralThesis,"['Segundo Reinado', 'Baianismo', 'Política da ...",['Não informado pela instituição'],por,A presente tese investiga o baianismo e suas i...,https://repositorio.ufba.br/handle/ri/36441,Não informado pela instituição,Não informado pela instituição,"CRUZ, Itan. Saraiva, Dantas e Cotegipe: baiani..."


Preparando o texto em turtle

In [5]:
# Prefixos
prefixos = """ @prefix ns: <http://www.w3.org/2003/06/sw-vocab-status/ns#> .
@prefix owl: <http://www.w3.org/2002/07/owl#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix bibo: <http://purl.org/ontology/bibo/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix vann: <http://purl.org/vocab/vann/> .
@prefix event: <http://purl.org/NET/c4dm/event.owl#> .
@prefix prism: <http://prismstandard.org/namespaces/1.2/basic/> .
@prefix terms: <http://purl.org/dc/terms/> .
@prefix schema: <http://schemas.talis.com/2005/address/schema#> .
@prefix status: <http://purl.org/ontology/bibo/status/> .
@prefix degrees: <http://purl.org/ontology/bibo/degrees/> .
@prefix stardog: <tag:stardog:api:> .
@base <http://www.w3.org/2002/07/owl#> .

"""

Conectando com o banco e adicionando as triplas

In [6]:
# Funcão que recebe os prefixos e triplas e as carrega à base de dados
def add_triplas_to_stardog(prefixos, triplas):

    # Incluindo prefixos às triplas
    triplas = prefixos + " " + triplas
    triplas = triplas.encode('utf-8')

    ### Connect to the Stardog database
    database_name = 'IndigenousSlavery'
    conn = stardog.Connection(database_name, **connection_details)

    conn.begin()
    conn.add(stardog.content.Raw(triplas, 'text/turtle'))
    conn.commit() # commit the transaction

Limpando e processando os dados das Teses

In [9]:
# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))

In [15]:
titulo = data['Título']
thesis_uri = data['Título'].apply(lambda x: process_uri(x))
#Alterando os anos com dois dígitos para 4 dígitos
data['Ano de defesa'][data['Ano de defesa'].apply(str).apply(len) != 4] = data['Ano de defesa'][data['Ano de defesa'].apply(str).apply(len) != 4].apply(lambda x: 2000 + x if x < 25 else 1900 + x)
ano = data['Ano de defesa']
link = data['Link de acesso']
referencia = data['Referência Bibliográfica']
resumo = data['Resumo']
resumo_pt = data['Resumo em Português']
resumo_en = data['Resumo em Inglês']
resumo_lang = data['Resumo'].apply(lambda x: detect(str(x) + "sem texto"))


C:\Users\facordei\AppData\Local\Temp\ipykernel_23732\3317037659.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['Ano de defesa'][data['Ano de defesa'].apply(str).apply(len) != 4] = data['Ano de defesa'][data['Ano de defesa'].apply(s

In [11]:
triplas = """ """

for m in range(len(thesis_uri)):
    tripla = """
    stardog:""" + thesis_uri[m] + """ rdf:type bibo:Thesis.
    stardog:""" + thesis_uri[m] + """ terms:title '""" + titulo[m].replace("'","").replace(u'\\', u' ') +  """'.
    stardog:""" + thesis_uri[m] + """ terms:created '""" + str(ano[m]) +  """'^^xsd:gYear.
    """
    if link[m].startswith("http"):
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:repository '""" + link[m].replace("'","").replace(u'\\', u' ') + """'. """

    if referencia[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:identifier '""" + referencia[m].replace("'","").replace(u'\\', u' ') + """'. """

    if resumo[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo[m]).replace("'","").replace(u'\\', u' ') + """'@""" + resumo_lang[m] + """. """
    
    if resumo_pt[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo_pt[m]).replace("'","").replace(u'\\', u' ') + """'@pt. """

    if resumo_en[m] != 'Não informado pela instituição':
        tripla = tripla + """stardog:""" + thesis_uri[m] + """ bibo:abstract '""" + str(resumo_en[m]).replace("'","").replace(u'\\', u' ') + """'@en. """

    triplas = triplas + " " + tripla

# stardog:""" + thesis_uri[m] + """ rdf:type owl:NamedIndividual.

In [ ]:
#add_triplas_to_stardog(prefixos, triplas)

Limpando dados e propriedades das pessoas ("Person") - Autores, Orientadores e Membros da Banca.

In [10]:
def clean_person(person_column):

    column_raw = person_column.apply(lambda x: x.split("',")).replace("['", "")

    column_raw_list = []
    for i in column_raw:
        for person in i:
            if person != '[]':
                person = person.replace("[", "").replace("]", "").replace("'", "").replace("", "").split(",")
                column_raw_list.append(person)

    column_raw_list = pd.Series(column_raw_list) 

    #Último Sobrenome   
    column_family_name = column_raw_list.apply(lambda x: x[0].strip().replace(u'\\', u' '))

    # Primeiros nomes
    column_given_name = column_raw_list.apply(lambda x: x[1].strip().replace(u'\\', u' ') if len(x) > 1 else "")

    # Unindo nome e sobrenome
    column_uri = (column_given_name + "_" + column_family_name) 

    #substituindo espaco por "_", retirando diacrítico e colocando em minúscula
    column_uri = column_uri.apply(lambda x: process_uri(x))

    return (column_raw_list, column_family_name, column_given_name, column_uri)  

In [11]:
# Processando colunas
autor_raw_list, autor_family_name, autor_given_name, autor_uri = clean_person(data["Autor(a)"])
orientador_raw_list, orientador_family_name, orientador_given_name, orientador_uri = clean_person(data['Orientadores'])
banca_raw_list, banca_family_name, banca_given_name, banca_uri = clean_person(data['Membros da banca'])

# Unindo as colunas com informacões de pessoas
family_name = pd.concat([autor_family_name, orientador_family_name, banca_family_name],  ignore_index=True)
given_name = pd.concat([autor_given_name, orientador_given_name, banca_given_name],  ignore_index=True)
name_uri = pd.concat([autor_uri, orientador_uri, banca_uri],  ignore_index=True)

# Removendo duplicados
name_uri = name_uri.drop_duplicates()
uniq_index = name_uri.index
family_name = family_name[uniq_index]
given_name = given_name[uniq_index]

In [12]:
triplas = """ """

for m in uniq_index:
    tripla = """
    stardog:""" + name_uri[m] + """ rdf:type foaf:Person.
    stardog:""" + name_uri[m] + """ rdfs:label '""" + family_name[m].replace("'","") + """, """+ given_name[m].replace("'","") + """'.
    stardog:""" + name_uri[m] + """ foaf:givenname '""" + given_name[m].strip().replace("'","") + """'.
    stardog:""" + name_uri[m] + """ foaf:family_name '""" + family_name[m].replace("'","") + """'.
    """

    triplas = triplas + " " + tripla

# stardog:""" + name_uri[m] + """ rdf:type owl:NamedIndividual.

In [ ]:
#add_triplas_to_stardog(prefixos, triplas)

In [16]:
# Adding autor relationship between thesis and its author.

triplas = """ """

for m in range(len(thesis_uri)):
    tripla = """
    stardog:""" + thesis_uri[m] + """ bibo:author stardog:""" + autor_uri[m] +  """."""
    triplas = triplas + " " + tripla

In [ ]:
#add_triplas_to_stardog(prefixos, triplas)

In [18]:
# Adding autor relationship between thesis and its advisors and opponents.

triplas = """ """

for m in range(len(thesis_uri)):
    person_uris = clean_person(data['Orientadores'][m:m+1])[3]
    for person_uri in person_uris:
        tripla = """
        stardog:""" + thesis_uri[m] + """ bibo:advisor stardog:""" + person_uri +  """."""
        triplas = triplas + " " + tripla

    person_uris = clean_person(data['Membros da banca'][m:m+1])[3]
    for person_uri in person_uris:
        tripla = """
        stardog:""" + thesis_uri[m] + """ bibo:opponent stardog:""" + person_uri +  """."""
        triplas = triplas + " " + tripla


In [ ]:
#add_triplas_to_stardog(prefixos, triplas)

Limpando dados e propriedades da instituicao de pesquisa

In [20]:
#Criando as triplas das instituicões
university = data[['Instituição de defesa', 'Sigla da instituição de defesa']].drop_duplicates(ignore_index=True)

triplas = """ """

for m in range(len(university)):
    tripla = """
    stardog:""" + university['Sigla da instituição de defesa'][m] + """ rdf:type bibo:University. 
    stardog:""" + university['Sigla da instituição de defesa'][m] + """ rdfs:label '""" + university['Instituição de defesa'][m] + """'.
    stardog:""" + university['Sigla da instituição de defesa'][m] + """ bibo:acronym '""" + university['Sigla da instituição de defesa'][m] + """'.
    """
    triplas = triplas + " " + tripla

# stardog:""" + university['Sigla da instituição de defesa'][m] + """ rdf:type owl:NamedIndividual.

In [ ]:
#add_triplas_to_stardog(prefixos, triplas)

In [22]:
# Criando as triplas dos departamentos e programas
departamento = data[data['Departamento da instituição de defesa'] != 'Não informado pela instituição']
departamento = departamento[['Sigla da instituição de defesa', 'Departamento da instituição de defesa']].drop_duplicates(ignore_index=True)
departamento['uri'] = departamento['Sigla da instituição de defesa'] + '_' + departamento['Departamento da instituição de defesa'].apply(process_uri)


programa = data[data['Programa de Pós-Graduação da instituição de defesa'] != 'Não informado pela instituição']
programa = programa[['Sigla da instituição de defesa', 'Programa de Pós-Graduação da instituição de defesa']].drop_duplicates(ignore_index=True)
programa['uri'] = programa['Sigla da instituição de defesa'] + '_' + programa['Programa de Pós-Graduação da instituição de defesa'].apply(process_uri)

triplas = """ """

for m in range(len(departamento)):
    tripla = """
    stardog:""" + departamento['uri'][m] + """ rdf:type bibo:Department. 
    stardog:""" + departamento['uri'][m] + """ rdfs:label '""" + departamento['Departamento da instituição de defesa'][m] + """'.
    stardog:""" + departamento['uri'][m] + """ <http://purl.obolibrary.org/obo/BFO_0000050> stardog:""" + departamento['Sigla da instituição de defesa'][m] + """.
    """
    triplas = triplas + " " + tripla

for m in range(len(programa)):
    tripla = """
    stardog:""" + programa['uri'][m] + """ rdf:type bibo:Program. 
    stardog:""" + programa['uri'][m] + """ rdfs:label '""" + programa['Programa de Pós-Graduação da instituição de defesa'][m] + """'.
    stardog:""" + programa['uri'][m] + """ <http://purl.obolibrary.org/obo/BFO_0000050> stardog:""" + programa['Sigla da instituição de defesa'][m] + """.
    """
    triplas = triplas + " " + tripla

# stardog:""" + departamento['uri'][m] + """ rdf:type owl:NamedIndividual.
# stardog:""" + programa['uri'][m] + """ rdf:type owl:NamedIndividual.

In [ ]:
#add_triplas_to_stardog(prefixos, triplas)

Conectando as teses às organizacoes (univeridades, departamentos e programas)

In [24]:
publishers = data[['Sigla da instituição de defesa', 'Departamento da instituição de defesa', 'Programa de Pós-Graduação da instituição de defesa']] 
publishers['dep_uri'] = publishers['Sigla da instituição de defesa'] + '_' + publishers['Departamento da instituição de defesa'].apply(process_uri)
publishers['program_uri'] = publishers['Sigla da instituição de defesa'] + '_' + publishers['Programa de Pós-Graduação da instituição de defesa'].apply(process_uri)


triplas = """ """

for m in range(len(thesis_uri)):
    if publishers['Sigla da instituição de defesa'][m] != 'Não informado pela instituição':
        tripla = """
        stardog:""" + thesis_uri[m] + """ terms:publisher stardog:""" + publishers['Sigla da instituição de defesa'][m] +  """."""
        triplas = triplas + " " + tripla

    if publishers['Departamento da instituição de defesa'][m] != 'Não informado pela instituição':
        tripla = """
        stardog:""" + thesis_uri[m] + """ terms:publisher stardog:""" + publishers['dep_uri'][m] +  """."""
        triplas = triplas + " " + tripla

    if publishers['Programa de Pós-Graduação da instituição de defesa'][m] != 'Não informado pela instituição':
        tripla = """
        stardog:""" + thesis_uri[m] + """ terms:publisher stardog:""" + publishers['program_uri'][m] +  """."""
        triplas = triplas + " " + tripla

C:\Users\facordei\AppData\Local\Temp\ipykernel_23732\1663232402.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publishers['dep_uri'] = publishers['Sigla da instituição de defesa'] + '_' + publishers['Departamento da instituição de defesa'].apply(process_uri)
C:\Users\facordei\AppData\Local\Temp\ipykernel_23732\1663232402.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  publishers['program_uri'] = publishers['Sigla da instituição de defesa'] + '_' + publishers['Programa de Pós-Graduação da instituiçã

In [ ]:
#add_triplas_to_stardog(prefixos, triplas)

Limpando os assuntos e áreas do conhecimento

In [26]:
# Filtrando as listas únicas (SET) com assuntos
# Inglês
assunto_en_v1 = data['Assuntos em inglês'].apply(lambda x: x.replace("'", "").replace("[", "").replace("]", "").split(",")).values

assunto_en_v2 = []
for i in assunto_en_v1:
    assunto_en_v2 = assunto_en_v2 + i

assunto_en_v3 = []
for i in assunto_en_v2:
    assunto_en_v3.append(i.strip())
assunto_en = set(assunto_en_v3)

In [27]:
#Português
assunto_pt_v1 = data['Assuntos em português'].apply(lambda x: x.replace("'", "").replace("[", "").replace("]", "").split(",")).values

assunto_pt_v2 = []
for i in assunto_pt_v1:
    assunto_pt_v2 = assunto_pt_v2 + i

assunto_pt_v3 = []
for i in assunto_pt_v2:
    assunto_pt_v3.append(i.strip())
assunto_pt = set(assunto_pt_v3)

In [28]:
# Funcao para encontrar o lemma da palavra em inglês e formar a URI do assunto

nlp = spacy.load("en_core_web_trf")

def lemma(word):
    doc = nlp(word)
    uri = ''
    for token in doc:
        uri = uri + token.lemma_ + ' '
    return (process_uri(uri))

In [60]:
# Funcao para traduzir os assuntos em português para inglês

# Add your key and endpoint
key = os.getenv('TRANSLATOR_KEY') #"<your-translator-key>"
endpoint = os.getenv('TRANSLATOR_ENDPOINT') #"https://api.cognitive.microsofttranslator.com"

# location, also known as region.
# required if you're using a multi-service or regional (not global) resource. It can be found in the Azure portal on the Keys and Endpoint page.
location = os.getenv('TRANSLATOR_LOCATION') #"<YOUR-RESOURCE-LOCATION>"

path = '/translate'
constructed_url = endpoint + path

params = {
    'api-version': '3.0',
    'from': 'pt',
    'to': ['en']
}

headers = {
    'Ocp-Apim-Subscription-Key': key,
    # location required if you're using a multi-service or regional (not global) resource.
    'Ocp-Apim-Subscription-Region': location,
    'Content-type': 'application/json',
    'X-ClientTraceId': str(uuid.uuid4())
}

# You can pass more than one object in body.
#body = [{
#    'text': 'I would really like to drive your car around the block a few times!'
#}]

def translate_pt_en(word):
    body = [{'text': word}]

    request = requests.post(constructed_url, params=params, headers=headers, json=body, verify=False)
    response = request.json()
    return (response[0]['translations'][0]['text'])
    #print(json.dumps(response, sort_keys=True, ensure_ascii=False, indent=4, separators=(',', ': ')))

In [47]:
# Criando um dicionário com assunto_URI e termos em inglês
uri_dic = {}

for word in assunto_en:
    assunto_uri = lemma(word)
    if assunto_uri not in uri_dic:
        uri_dic[assunto_uri] = {'en': [word], 'pt': []}
    else:
        uri_dic[assunto_uri]['en'] = uri_dic[assunto_uri]['en'] + [word]

In [ ]:
#Removing InsecureRequestWarning
import urllib3
urllib3.disable_warnings()

# Criando um dicionário com assunto_URI e termos em português
#n = 0
#for word in assunto_pt:
for n in range(29400, len(list(assunto_pt))):
    #n = n + 1
    if n % 100 == 0:
        print(n)
        time.sleep(2.5)
    word = list(assunto_pt)[n]
    assunto_translate = translate_pt_en(word)
    assunto_uri = lemma(assunto_translate)
    if assunto_uri not in uri_dic:
        uri_dic[assunto_uri] = {'en': [assunto_translate], 'pt': [word]}
    else:
        uri_dic[assunto_uri]['en'] = list(set(uri_dic[assunto_uri]['en'] + [assunto_translate]))
        uri_dic[assunto_uri]['pt'] = uri_dic[assunto_uri]['pt'] + [word]

19200
19300
19400
19500
19600
19700
19800
19900
20000
20100
20200
20300
20400
20500
20600
20700
20800
20900
21000
21100
21200
21300
21400
21500
21600
21700
21800
21900
22000
22100
22200
22300
22400
22500
22600
22700
22800
22900
23000
23100
23200
23300
23400
23500
23600
23700
23800
23900
24000
24100
24200
24300
24400
24500
24600
24700
24800
24900
25000
25100
25200
25300
25400
25500
25600
25700
25800
25900
26000
26100
26200
26300
26400
26500
26600
26700
26800
26900
27000
27100
27200
27300
27400
27500
27600
27700
27800
27900
28000
28100
28200
28300
28400
28500
28600
28700
28800
28900
29000
29100
29200
29300
29400


ConnectionError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [70]:
uri_dic

{'': {'en': ['', '.'], 'pt': ['', '', '.']},
 'life_project': {'en': ['Life project', 'Life projects'],
  'pt': ['Life projects', 'Projetos de vida']},
 'history_of_africa_': {'en': ['history of africa;'], 'pt': []},
 'lgbtqiaphobia': {'en': ['LGBTQIA+phobia'], 'pt': ['LGBTQIA+fobia']},
 'geography_teaching_in_eja': {'en': ['Geography teaching in EJA'], 'pt': []},
 'health_personnel': {'en': ['Health personnel'], 'pt': []},
 'hydrocarbon': {'en': ['Hydrocarbon'], 'pt': ['Hidrocarbonetos']},
 'text_production': {'en': ['Text production'], 'pt': ['Produção de textos']},
 'stereotyping': {'en': ['Stereotyping'], 'pt': []},
 'latin_american_social_movement': {'en': ['Latin American social movements'],
  'pt': []},
 'urban_renewal': {'en': ['Urban renewal'], 'pt': []},
 'security_and_defense_in_south_america': {'en': ['Security and defense in south america'],
  'pt': []},
 'human_capital': {'en': ['Human capital'],
  'pt': ['Human capital', 'Human capital', 'Capital humano']},
 'protocol_':

In [71]:
with open('data/assunto_dict.json', 'w') as fp:
    json.dump(uri_dic, fp)

In [56]:
# A lista em de assuntos em português, possui muito termos em inglês porque os cadastros estao misturados. 
# Vamos verificar se um termo em portugês já existe na lista em ingles e exclui-los

for uri in uri_dic:
    for assunto_pt in uri_dic[uri]['pt']:
        if assunto_pt in uri_dic[uri]['en']:
            uri_dic[uri]['pt'].remove(assunto_pt)

# Excluindo do dicionário os termos ' ' e 'Não informado pela instituição'.

del uri_dic['']
del uri_dic['_']
del uri_dic[process_uri('Não informado pela instituição')]
del uri_dic[process_uri('Not informed by the institution')]

# save dict
with open('data/assunto_dict.json', 'w') as fp:
    json.dump(uri_dic, fp)

In [58]:
len(uri_dic)

30824

In [59]:
# Carregando as triplas dos assuntos na base de dados 

uri_dic_keys = list(uri_dic.keys())

triplas = """ """
for n in range(0, len(uri_dic_keys)):

    tripla = """
    stardog:""" + uri_dic_keys[n] + """ rdf:type bibo:Subject."""
    triplas = triplas + " " + tripla

    for label_pt in uri_dic[uri_dic_keys[n]]['pt']:
        tripla = """stardog:""" + uri_dic_keys[n] + """ rdfs:label '""" + str(label_pt).replace("'","").replace(u'\\', u' ') + """'@pt. """
        triplas = triplas + " " + tripla

    for label_en in uri_dic[uri_dic_keys[n]]['en']:
        tripla = """stardog:""" + uri_dic_keys[n] + """ rdfs:label '""" + str(label_en).replace("'","").replace(u'\\', u' ') + """'@en. """
        triplas = triplas + " " + tripla

    if n%1000 == 0: 
        print (n, " URIs carregadas")
        add_triplas_to_stardog(prefixos, triplas)
        triplas = """ """

print (n, " URIs carregadas")
add_triplas_to_stardog(prefixos, triplas)

0  URIs carregadas
1000  URIs carregadas
2000  URIs carregadas
3000  URIs carregadas
4000  URIs carregadas
5000  URIs carregadas
6000  URIs carregadas
7000  URIs carregadas
8000  URIs carregadas
9000  URIs carregadas
10000  URIs carregadas
11000  URIs carregadas
12000  URIs carregadas
13000  URIs carregadas
14000  URIs carregadas
15000  URIs carregadas
16000  URIs carregadas
17000  URIs carregadas
18000  URIs carregadas
19000  URIs carregadas
20000  URIs carregadas
21000  URIs carregadas
22000  URIs carregadas
23000  URIs carregadas
24000  URIs carregadas
25000  URIs carregadas
26000  URIs carregadas
27000  URIs carregadas
28000  URIs carregadas
29000  URIs carregadas
30000  URIs carregadas
30823  URIs carregadas


Conectando as teses aos assuntos

In [60]:
# criando dicionario com label apontando para as uri
# portugues
label_uri_pt = {}
for uri in uri_dic:
    for assunto_pt in uri_dic[uri]['pt']:
        label_uri_pt[assunto_pt] = uri

with open('data/assunto_uri_pt.json', 'w') as fp:
    json.dump(label_uri_pt, fp)

# inglês
label_uri_en = {}
for uri in uri_dic:
    for assunto_en in uri_dic[uri]['en']:
        label_uri_en[assunto_en] = uri

with open('data/assunto_uri_en.json', 'w') as fp:
    json.dump(label_uri_en, fp)

In [64]:
# Abrindo os dicionários salvos anteriormente

with open('data/assunto_dict.json') as fp:
    uri_dic = json.load(fp)
    
with open('data/assunto_uri_pt.json') as fp:
    label_uri_pt = json.load(fp)

with open('data/assunto_uri_en.json') as fp:
    label_uri_en = json.load(fp)

In [76]:
# Adding relationship between thesis and its subjects.

triplas = """ """

for m in range(1, len(thesis_uri)):
    subj_uri = []
    for sub in assunto_pt_v1[m]:
        try:
            subj_uri.append(label_uri_en[sub.strip()])
        except:
            try:
                subj_uri.append(label_uri_pt[sub.strip()])
            except:
                pass
    for sub in assunto_en_v1[m]:
        try:
            subj_uri.append(label_uri_en[sub.strip()])
        except:
            try:
                subj_uri.append(label_uri_pt[sub.strip()])
            except:
                pass
            
    for sub in set(subj_uri):
        
        tripla = """stardog:""" + thesis_uri[m] + """ terms:subject stardog:""" + sub +  """."""
        triplas = triplas + " " + tripla

    if m%1000 == 0: 
        print (m, " teses carregadas")
        add_triplas_to_stardog(prefixos, triplas)
        triplas = """ """

print (m, " URIs carregadas")
add_triplas_to_stardog(prefixos, triplas)

1000  teses carregadas
2000  teses carregadas
3000  teses carregadas
4000  teses carregadas
5000  teses carregadas
6000  teses carregadas
7000  teses carregadas
8000  teses carregadas
9000  teses carregadas
10000  teses carregadas
11000  teses carregadas
12000  teses carregadas
13000  teses carregadas
14000  teses carregadas
14410  URIs carregadas


Processando as àreas de conhecimento

In [77]:
# Excluindo as áreas não informadas pela instituicão
AC = data['Área do conhecimento CNPq'][ data['Área do conhecimento CNPq'] != 'Não informado pela instituição']
#Identificando as sequencias de areas únicas (todo um ramo da taxonomia)
AC = AC.apply(lambda x: x.split('||'))
AC_list = []
for area in AC:
    AC_list = AC_list + area
AC_list = list(set(AC_list))

In [78]:
# Lista para receber o ramo da taxonomia, já processada como uri
AC_uri_list = []
# Dicionário para converter em URI para label
AC_uri_dic = {}

for areas in AC_list:
    uri_area = []
    for area in areas.split('::'):
        area_translate = translate_pt_en(area)
        uri_a = lemma(area_translate)
        #uri_a = process_uri(area)
        uri_area.append(uri_a)
        if uri_a not in AC_uri_dic:
            AC_uri_dic[uri_a] = [area]
        else:
            AC_uri_dic[uri_a] = list(set(AC_uri_dic[uri_a] + [area]))
    AC_uri_list.append(uri_area)

# Removendo o termo 'cnpq' e 'accnpq' do dicionário e das listas
del AC_uri_dic['cnpq']

new_AC_uri_list = []
for l in AC_uri_list:
    new_AC_uri_list.append([i for i in l if i != 'cnpq'])
AC_uri_list = new_AC_uri_list

del AC_uri_dic['accnpq']

new_AC_uri_list = []
for l in AC_uri_list:
    new_AC_uri_list.append([i for i in l if i != 'accnpq'])
AC_uri_list = new_AC_uri_list

# Montando as tuplas com "area do conhecimento" has_part "outra area do conhecimento".

tupla_has_part = []
for areas in AC_uri_list:
    if len(areas) > 1:
        for n in range(len(areas)-1):
            tupla_has_part.append((areas[n], areas[n+1]))

tupla_has_part = list(set(tupla_has_part))

In [79]:
# Carregando as triplas das áreas de conhecimento na base de dados 

uri_dic_keys = list(AC_uri_dic.keys())

triplas = """ """
for n in range(0, len(uri_dic_keys)):

    tripla = """
    stardog:""" + uri_dic_keys[n] + """ rdf:type bibo:CNPQ."""
    triplas = triplas + " " + tripla

    for label_pt in AC_uri_dic[uri_dic_keys[n]]:
        tripla = """stardog:""" + uri_dic_keys[n] + """ rdfs:label '""" + str(label_pt).replace("'","").replace(u'\\', u' ') + """'@pt. """
        triplas = triplas + " " + tripla
        
    if n%100 == 0: 
        print (n, " URIs carregadas")
        add_triplas_to_stardog(prefixos, triplas)
        triplas = """ """

print (n, " URIs carregadas")
add_triplas_to_stardog(prefixos, triplas)

0  URIs carregadas
100  URIs carregadas
200  URIs carregadas
300  URIs carregadas
400  URIs carregadas
500  URIs carregadas
561  URIs carregadas


In [80]:
# Carregando as triplas "hasp_part" das áreas de conhecimento na base de dados 
triplas = """ """
for n in tupla_has_part:
    tripla = """
    stardog:""" + n[0] + """ <http://purl.obolibrary.org/obo/BFO_0000051> stardog:""" + n[1] + """."""
    triplas = triplas + " " + tripla
    

In [81]:
#add_triplas_to_stardog(prefixos, triplas)

Conectando as teses às áreas do conhecimento

In [82]:
# criando dicionario com label das áreas do conhecimento apontando para as uri
label_area_uri = {}
for uri in AC_uri_dic:
    for area in AC_uri_dic[uri]:
        label_area_uri[area] = uri

In [88]:
# Adding relationship between thesis and its subjects.

triplas = """ """

for m in AC.index:
    for areas in AC[m]:
        for area in areas.split('::'):
            try:
                tripla = """stardog:""" + thesis_uri[m] + """ terms:subject stardog:""" + label_area_uri[area] +  """."""
                triplas = triplas + " " + tripla
            except:
                pass

    if m%1000 == 0: 
        print (m, " URIs carregadas")
        add_triplas_to_stardog(prefixos, triplas)
        triplas = """ """

print (m, " URIs carregadas")
add_triplas_to_stardog(prefixos, triplas)

3000  URIs carregadas
4000  URIs carregadas
7000  URIs carregadas
8000  URIs carregadas
9000  URIs carregadas
13000  URIs carregadas
14000  URIs carregadas
14401  URIs carregadas


Carregando as triplas conectando as teses aos degree

In [89]:
triplas = """
<http://purl.org/ontology/bibo/degrees/phd> rdf:type <http://purl.org/ontology/bibo/ThesisDegree>.
<http://purl.org/ontology/bibo/degrees/phd> rdfs:label 'PhD degree'@en.
<http://purl.org/ontology/bibo/degrees/phd> rdfs:label 'Doutorado'@pt.
<http://purl.org/ontology/bibo/degrees/master> rdf:type <http://purl.org/ontology/bibo/ThesisDegree>.
<http://purl.org/ontology/bibo/degrees/master> rdfs:label 'Master degree'@en.
<http://purl.org/ontology/bibo/degrees/master> rdfs:label 'Mestrado'@pt."""

add_triplas_to_stardog(prefixos, triplas)

In [90]:
# Adding relationship between thesis and degree.

triplas = """ """

for m in range(0, len(thesis_uri)):
    if data['Tipo de documento'][m] == "masterThesis":
        tripla = """stardog:""" + thesis_uri[m] + """ bibo:degree <http://purl.org/ontology/bibo/degrees/master>."""
        triplas = triplas + " " + tripla
    if data['Tipo de documento'][m] == "doctoralThesis":
        tripla = """stardog:""" + thesis_uri[m] + """ bibo:degree <http://purl.org/ontology/bibo/degrees/phd>."""        
        triplas = triplas + " " + tripla

In [91]:
add_triplas_to_stardog(prefixos, triplas)